In [9]:
import tensorflow as tf
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
from  tensorflow.keras import layers
import time
from sklearn.utils import shuffle
from IPython import display
from imutils import build_montages
import cv2

In [10]:
# Load dataset...

((trainX, _), (testX, _)) = tf.keras.datasets.fashion_mnist.load_data()
#((trainX, _), (testX, _)) = tf.keras.datasets.mnist.load_data()

#combine train & test images to use all the dataset for learning process.
trainImages = np.concatenate([trainX, testX])
trainImages = np.expand_dims(trainImages, axis=-1)

# normalization of images between -1 and 1 
trainImages  = (trainImages.astype("float") - 127.5) / 127.5

buffer_size = trainImages.shape[0]
batch_size = 256 

#Noise dimention (100,) is used to make noise dataset (noise images) to give to Generator 
noise_dim = 100  


In [28]:

class GAN:
  
#  This is the main class of GAN consists of several methods
  def __init__(self, buffersize, batchsize=256, noisedimention=100):
    self.GAN = None
    self.Generator= None
    self.Discriminator = None
    
    self.BUFFER_SIZE = buffersize
    self.BATCH_SIZE = batchsize
    self.Batchperepochs = np.floor(self.BUFFER_SIZE / self.BATCH_SIZE).astype('int32')
    self.EPOCHS = 50
    self.noise_dim = noisedimention
    self.lr = 2e-4
    self.create_GAN()
    print("A GAN with one generator and one discriminator is created")


  def make_generator(self,dim=7, depth=64, inputDim=100, outputDim=512, channels=1):
    '''
    Make an image Generator  
    '''
    inputShape = (dim, dim, depth)
    chanDim = -1
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(input_dim=inputDim, units=outputDim, use_bias= False, input_shape=(100,)))
    model.add(layers.Activation("relu"))
    model.add(layers.BatchNormalization())

    model.add(layers.Dense(7 * 7 * 64))
    model.add(layers.Activation("relu"))
    model.add(layers.BatchNormalization())
    
    model.add(layers.Reshape(inputShape))
    model.add(layers.Conv2DTranspose(32, (5, 5), strides=(2, 2),padding="same"))
    model.add(layers.Activation("relu"))
    model.add(layers.BatchNormalization(axis=chanDim))

    model.add(layers.Conv2DTranspose(channels, (5, 5), strides=(2, 2),padding="same"))
    model.add(layers.Activation("tanh"))

    return model

    
  def make_discriminator(self, alpha=0.2):
    '''
    Make a Discriminator
    '''
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(32, 5 , strides=(2,2), use_bias= False, padding='same', input_shape=[28,28,1]))
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())

    model.add(layers.Flatten())
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(alpha=alpha))

    model.add(layers.Dense(1))
    model.add(layers.Activation("sigmoid"))

    return model

  def create_GAN(self):
    
    self.Generator = self.make_generator()
    self.Discriminator = self.make_discriminator()
    self.Discriminator.compile(optimizer = tf.keras.optimizers.legacy.Adam(0.001,beta_1=0.5, decay=self.lr / (self.EPOCHS+50)),loss="binary_crossentropy")
    self.Discriminator.trainable = False

    ganinput = layers.Input((100,))
    ganoutput = self.Discriminator( self.Generator(ganinput))
    self.GAN = tf.keras.Model(ganinput, ganoutput)
    self.GAN.compile(optimizer= tf.keras.optimizers.legacy.Adam(lr=self.lr, beta_1=0.5, decay=self.lr / (self.EPOCHS+50)),loss="binary_crossentropy")
    display.clear_output(wait=True) 


  def train(self, train_images, epochs):
    print("Start training...")
    benchmarkNoise = tf.random.normal((self.BATCH_SIZE, self.noise_dim))
    self.EPOCHS = epochs
    
    for epoch in range(0,epochs):
      for j in range(0, self.Batchperepochs):
        p = None
        noises = tf.random.normal((self.BATCH_SIZE, self.noise_dim))
        generated_images= self.Generator.predict(noises, verbose=0)

        imgs = np.concatenate((train_images[j * self.BATCH_SIZE: (j+1) * self.BATCH_SIZE], generated_images ))
        #imgs = np.concatenate((batch, generated_images ))
        labels = ([1] * self.BATCH_SIZE) + ([0] * self.BATCH_SIZE)
        labels = np.reshape(labels, (-1,))
        (imgs, labels) = shuffle(imgs, labels)

        discloss = self.Discriminator.train_on_batch(imgs,labels)

        noises = tf.random.normal((self.BATCH_SIZE, self.noise_dim))
        fakelabels = ([1] * self.BATCH_SIZE)
        fakelabels = np.reshape(fakelabels, (-1,))

        GANloss = self.GAN.train_on_batch(noises, fakelabels)

        if j== self.Batchperepochs - 1: 
          p =  ["epoch_{}_2_output.png".format(str(epoch + 1).zfill(4))]
        elif j == np.int32(self.Batchperepochs/2):
          p =  ["epoch_{}_1_output.png".format(str(epoch + 1).zfill(4))]
                
        if p is not None:
          print("Step {}_{}: Discriminator_loss={:.6f}, "
				    "GAN_loss={:.6f}".format(epoch + 1, j,
				    	discloss, GANloss))
          images = self.Generator.predict(benchmarkNoise)
          images = ((images * 127.5) + 127.5).astype("uint8")
          images = np.repeat(images, 3, axis=-1)
          vis = build_montages(images, (28, 28), (16, 16))[0]
          p = os.path.sep.join(p)
          cv2.imwrite(p, vis)

    plt.imshow(vis)

  def predict(self, noises):
    predicted_images = self.Generator.predict(noises , verbose=0)
    predicted_images = ((predicted_images * 127.5) + 127.5).astype("uint8")
    predicted_images = np.repeat(predicted_images, 3, axis=-1)
    vis = build_montages(predicted_images, (28, 28), (16, 16))[0]
    return vis
    



In [ ]:
# Make an instance of our GAN class (our model)
my_model = GAN(buffer_size, batch_size)


In [ ]:
# Train the model...
my_model.train(trainImages, 1)

In [ ]:
# Test the model by giving random noise
res = my_model.predict(tf.random.normal((batch_size,100)))
plt.imshow(res)